#### Qlunc tutorial by Francisco Costa
# **Tutorial 1**
## Qlunc step by step

This tutorial aims to facilitate the introduction to Qlunc. 
Will go through the code and create a lidar device with its modules and components. Will ask for uncertainties either lidar general one or component specific uncertainty. We will see some graphical interesting results. Will see how to access design lidar data.

Importing some packages is always needed:

In [1]:
import os
os.chdir('../')
# importing  uncertainty functions
import UQ_Functions.UQ_Photonics_Classes as uphc,UQ_Functions.UQ_Optics_Classes as uopc, UQ_Functions.UQ_Power_Classes as upwc,UQ_Functions.UQ_Lidar_Classes as ulc
from Utils.Qlunc_ImportModules import *

### Qlunc inputs:  

A [.yml template](https://github.com/PacoCosta/Qlunc/blob/Qlunc-V0.9/Main/Template_yaml_inputs_file.yml) is included in the repository. To accomplish this tutorial a yaml file is already filled. 

The code below imports the `yaml` file.

In [2]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is 0C54-9785

 Directory of C:\Users\fcosta\SWE_LOCAL\GIT_Qlunc

28.06.2021  10:24    <DIR>          .
28.06.2021  10:24    <DIR>          ..
11.03.2021  10:53    <DIR>          .github
17.06.2021  18:32               247 .gitignore
10.03.2021  17:46    <DIR>          Docs
28.06.2021  10:34               473 environment.yml
30.03.2021  02:01             1.566 LICENSE
02.07.2021  16:10    <DIR>          Main
28.06.2021  10:31    <DIR>          metadata
12.03.2021  00:06    <DIR>          Pictures_repo_
02.07.2021  16:13    <DIR>          Projects
29.06.2021  23:34             8.401 readme.md
11.03.2021  11:13    <DIR>          Sphinx4Qlunc
01.07.2021  11:15    <DIR>          TestFiles_Qlunc
28.06.2021  10:28    <DIR>          Tutorials
01.07.2021  11:52    <DIR>          UnderDevelopment
02.07.2021  16:37    <DIR>          UQ_Functions
02.07.2021  11:54    <DIR>          Utils
30.03.2021  02:01                93 __init__.py
30.03.20

In [3]:
# Obtain data from .yaml file:
with open (r'./Tutorials/yaml_inputs_file_1.yml','r') as file:
    Qlunc_yaml_inputs={}
    docs = yaml.load_all(file, Loader=yaml.FullLoader)
    for doc in docs:      
        for k, v in doc.items():     
            Qlunc_yaml_inputs.setdefault(k,v)

We obtain a dictionary, a well-known python object, with all the data introduced in the `yaml` file. Let´s have a look to the data:

In [4]:
Qlunc_yaml_inputs

{'Main_directory': './Tutorials',
 'Components': {'Scanner': {'Name': 'Scanner_Yaml',
   'Origin': [0, 0, 0],
   'Pattern': 'None',
   'Sample rate': 0,
   'Type': 'VAD',
   'stdv focus distance': 1,
   'stdv Cone angle': 1,
   'stdv Azimuth': 1,
   'Focus distance': [40],
   'Cone angle': [20],
   'Azimuth': [0, 360, 8],
   'x': [20, 20, 20],
   'y': [0, 0, 0],
   'z': [20, 0, -20],
   'Uncertainty function': 'uopc.UQ_Scanner'},
  'Optical Circulator': {'Name': 'OC_Yaml',
   'Insertion loss': 2.1,
   'Uncertainty function': 'uopc.UQ_OpticalCirculator'},
  'Optical Amplifier': {'Name': 'OA_Yaml',
   'Optical amplifier noise figure': 1,
   'Optical amplifier gain': 30,
   'Uncertainty function': 'uphc.UQ_Optical_amplifier'},
  'Photodetector': {'Name': 'Photodetector_Yaml',
   'Photodetector BandWidth': 380000000.0,
   'Load resistor': 50,
   'Photodetector efficiency': 0.85,
   'Dark current': 5e-09,
   'Photodetector signalP': 0.001,
   'Power interval': [0, 1000, 0.001],
   'Gain TIA

Now we have the component inputs, let's create them.

### **Creating the lidar classes**:

We create the python classes by running the Qlunc Core: `Qlunc_Classes.py`

In [5]:
# Execute Qlunc_Classes.py (creating classes for lidar 'objects'):
exec(open(Qlunc_yaml_inputs['Main_directory']+'/WorkingExample_Tutorial.py').read())

In [6]:
!dir
Qlunc_yaml_inputs['Main_directory']

 Volume in drive C has no label.
 Volume Serial Number is 0C54-9785

 Directory of C:\Users\fcosta\SWE_LOCAL\GIT_Qlunc

28.06.2021  10:24    <DIR>          .
28.06.2021  10:24    <DIR>          ..
11.03.2021  10:53    <DIR>          .github
17.06.2021  18:32               247 .gitignore
10.03.2021  17:46    <DIR>          Docs
28.06.2021  10:34               473 environment.yml
30.03.2021  02:01             1.566 LICENSE
02.07.2021  16:10    <DIR>          Main
28.06.2021  10:31    <DIR>          metadata
12.03.2021  00:06    <DIR>          Pictures_repo_
02.07.2021  16:13    <DIR>          Projects
29.06.2021  23:34             8.401 readme.md
11.03.2021  11:13    <DIR>          Sphinx4Qlunc
01.07.2021  11:15    <DIR>          TestFiles_Qlunc
28.06.2021  10:28    <DIR>          Tutorials
01.07.2021  11:52    <DIR>          UnderDevelopment
02.07.2021  16:37    <DIR>          UQ_Functions
02.07.2021  11:54    <DIR>          Utils
30.03.2021  02:01                93 __init__.py
30.03.20

'./Tutorials'

The main directory is taken from the yaml file, as well as the rest of the lidar parameters, including plotting flags which select what we want to plot:

### **Creating the lidar digital twin**:

We can now begin building up the lidar device and we will do it modularly. 
#### Scanner digital twin:
Will begin with the optics module and want to include in here two components: a scanner and an optical circulator, wich characteristics has been introduced in the yaml file.

Then, we can instance the class _scanner_ and the class _optical_circulator_ to build up an _optics_module_ wich, in turn, is created by instantiating the class _optics_module_.

Let's instantiate the class _scanner_ to "create" the scanner object:

In [7]:
# Instantiating scanner class to create a Scanner virtual-twin

Scanner           = scanner(name            = Qlunc_yaml_inputs['Components']['Scanner']['Name'],           # Introduce your scanner name.
                            scanner_type    = Qlunc_yaml_inputs['Components']['Scanner']['Type'],
                            origin          = Qlunc_yaml_inputs['Components']['Scanner']['Origin'],         # Origin (coordinates of the lidar deployment).
                            pattern         = Qlunc_yaml_inputs['Components']['Scanner']['Pattern'],
                            sample_rate     = Qlunc_yaml_inputs['Components']['Scanner']['Sample rate'],    # for now introduce it in [degrees].
                           
                           # This values for focus distance, cone_angle and azimuth define a typical VAD scanning sequence:
                            azimuth         = np.array(np.arange(Qlunc_yaml_inputs['Components']['Scanner']['Azimuth'][0],                                                  
                                                                 Qlunc_yaml_inputs['Components']['Scanner']['Azimuth'][1],
                                                                 Qlunc_yaml_inputs['Components']['Scanner']['Azimuth'][2])), # Azimuth angle in [degrees].
                            focus_dist      = np.tile(Qlunc_yaml_inputs['Components']['Scanner']['Focus distance'],(1,len(np.arange(Qlunc_yaml_inputs['Components']['Scanner']['Azimuth'][0],                                                  
                                                                 Qlunc_yaml_inputs['Components']['Scanner']['Azimuth'][1],
                                                                 Qlunc_yaml_inputs['Components']['Scanner']['Azimuth'][2]))))[0],   # Focus distance in [meters]                                        
                            cone_angle      = np.tile(Qlunc_yaml_inputs['Components']['Scanner']['Cone angle'],(1,len(np.arange(Qlunc_yaml_inputs['Components']['Scanner']['Azimuth'][0],                                                  
                                                                 Qlunc_yaml_inputs['Components']['Scanner']['Azimuth'][1],
                                                                 Qlunc_yaml_inputs['Components']['Scanner']['Azimuth'][2]))))[0],   # Cone angle in [degrees].
                            x               = np.array(Qlunc_yaml_inputs['Components']['Scanner']['x']),
                            y               = np.array(Qlunc_yaml_inputs['Components']['Scanner']['y']),
                            z               = np.array(Qlunc_yaml_inputs['Components']['Scanner']['z']),
                            stdv_focus_dist = Qlunc_yaml_inputs['Components']['Scanner']['stdv focus distance'],                 # Focus distance standard deviation in [meters].
                            stdv_cone_angle = Qlunc_yaml_inputs['Components']['Scanner']['stdv Cone angle'],                 # Cone angle standard deviation in [degrees].
                            stdv_azimuth    = Qlunc_yaml_inputs['Components']['Scanner']['stdv Azimuth'],                 # Azimuth angle standard deviation in [degrees].
                            unc_func        = uopc.UQ_Scanner) #eval(Qlunc_yaml_inputs['Components']['Scanner']['Uncertainty function']) )    # here you put the function describing your scanner uncertainty. 



Created new scanner: Scanner_Yaml


The label "Yaml" in the name of the device informs us that the data was taken from a yaml file and not manually, instantiating the class in question.

#### Optical circulator digital twin:
Then we can create the optical circulator by instantiating the optical_circulator class as follows:

In [8]:
#Optical Circulator:

Optical_circulator = optical_circulator (name           = Qlunc_yaml_inputs['Components']['Optical Circulator']['Name'],
                                         insertion_loss = Qlunc_yaml_inputs['Components']['Optical Circulator']['Insertion loss'],              
                                         unc_func       = eval(Qlunc_yaml_inputs['Components']['Optical Circulator']['Uncertainty function']))  

Created new optical circulator: OC_Yaml


#### Optics module digital twin: 
Then we create the optics module by puting all optic components created so far (_Scanner_ and _Optical_circulator_) together

In [9]:
# Optics Module:

Optics_Module =  optics (name               = Qlunc_yaml_inputs['Modules']['Optics Module']['Name'],     # Introduce your Optics Module name.
                          scanner            = eval(Qlunc_yaml_inputs['Modules']['Optics Module']['Scanner']),             # Scanner instance (in this example "Scanner") or "None". "None" means that you don´t want to include Scanner in Optics Module, either in uncertainty calculations.
                          optical_circulator = eval(Qlunc_yaml_inputs['Modules']['Optics Module']['Optical circulator']),  # Optical Circulator instance (in this example "Optical_circulator") or "None". "None" means that you don´t want to include Optical circulator in Optics Module, either in uncertainty calculations.
                          laser              = eval(Qlunc_yaml_inputs['Modules']['Optics Module']['Laser']),
                          unc_func           = eval(Qlunc_yaml_inputs['Modules']['Optics Module']['Uncertainty function']))

Created new optic module: Optics_module_Yaml


In [10]:
Qlunc_yaml_inputs['Modules']['Optics Module']['Scanner']

'Scanner'

We have already created the optics module. Following the same method we can create the rest of the components and modules by instantiating their corresponding classes.

#### Optical amplifier digital twin:
Let's create the photonics module. First we do instantiate its components namely, photodetector and optical amplifier.

In [11]:
# Instantiating optical amplifier:
Optical_Amplifier = optical_amplifier(name     = Qlunc_yaml_inputs['Components']['Optical Amplifier']['Name'],        # Introduce your scanner name.
                                      NoiseFig = Qlunc_yaml_inputs['Components']['Optical Amplifier']['Optical amplifier noise figure'],          # In [dB]. Can introduce it as a table from manufactures (in this example the data is taken from Thorlabs.com, in section EDFA\Graps) or introduce a single well-known value
                                      OA_Gain  = Qlunc_yaml_inputs['Components']['Optical Amplifier']['Optical amplifier gain'],                         # In [dB]. (in this example the data is taken from Thorlabs.com, in section EDFA\Specs)
                                      unc_func = eval(Qlunc_yaml_inputs['Components']['Optical Amplifier']['Uncertainty function']))  # Function describing Optical Amplifier uncertainty. Further informaion in "UQ_Photonics_Classes.py" comments.


Created new optical amplifier: OA_Yaml


#### Photodetector digital twin:

In [12]:
# Instantiating Photodetector:
Photodetector    = photodetector(name             = Qlunc_yaml_inputs['Components']['Photodetector']['Name'],               # Introduce your photodetector name.
                                 Photo_BandWidth  = Qlunc_yaml_inputs['Components']['Photodetector']['Photodetector BandWidth'],                  # In[]. Photodetector bandwidth
                                 Load_Resistor    = Qlunc_yaml_inputs['Components']['Photodetector']['Load resistor'],                     # In [ohms]
                                 Photo_efficiency = Qlunc_yaml_inputs['Components']['Photodetector']['Photodetector efficiency'],                    # Photodetector efficiency [-]
                                 Dark_Current     = Qlunc_yaml_inputs['Components']['Photodetector']['Dark current'],                   #  In [A]. Dark current in the photodetector.
                                 Photo_SignalP    = Qlunc_yaml_inputs['Components']['Photodetector']['Photodetector signalP'],
                                 Power_interval   = np.array(np.arange(Qlunc_yaml_inputs['Components']['Photodetector']['Power interval'][0],
                                                                       Qlunc_yaml_inputs['Components']['Photodetector']['Power interval'][1],
                                                                       Qlunc_yaml_inputs['Components']['Photodetector']['Power interval'][2])),#np.arange(Qlunc_yaml_inputs['Components']['Photodetector']['Power interval']), # In [w]. Power interval for the photodetector domain in photodetector SNR plot. 
                                 Gain_TIA         = Qlunc_yaml_inputs['Components']['Photodetector']['Gain TIA'],                    # In [dB]. If there is a transimpedance amplifier.
                                 V_Noise_TIA      = Qlunc_yaml_inputs['Components']['Photodetector']['V Noise TIA'],                 # In [V]. If there is a transimpedance amplifier.
                                 unc_func         = eval(Qlunc_yaml_inputs['Components']['Photodetector']['Uncertainty function']))  # Function describing Photodetector uncertainty. Further informaion in "UQ_Photonics_Classes.py" comments.


Created new photodetector: Photodetector_Yaml


#### Photonics module digital twin:
And finally the Photonics module:

In [13]:
# Instantiating Photonics module:
Photonics_Module = photonics(name              = Qlunc_yaml_inputs['Modules']['Photonics Module']['Name'],        # Introduce your Photonics module name
                             photodetector     = eval(Qlunc_yaml_inputs['Modules']['Photonics Module']['Photodetector']),             # Photodetector instance (in this example "Photodetector") or "None". "None" means that you don´t want to include photodetector in Photonics Module, either in uncertainty calculations.
                             optical_amplifier = eval(Qlunc_yaml_inputs['Modules']['Photonics Module']['Optical amplifier']),         # Scanner instance (in this example "OpticalAmplifier") or "None". "None" means that you don´t want to include Optical Amplifier in Photonics Module, either in uncertainty calculations.
                             unc_func          = eval(Qlunc_yaml_inputs['Modules']['Photonics Module']['Uncertainty function']))


Created new photonic module: Photonics_module_Yaml


#### Lidar general inputs:

In [14]:
# Instantiating lidar general inputs
Lidar_inputs = lidar_gral_inp(name        = Qlunc_yaml_inputs['Components']['Lidar general inputs']['Name'],
                              wave        = Qlunc_yaml_inputs['Components']['Lidar general inputs']['Wavelength'],
                              yaw_error   = Qlunc_yaml_inputs['Components']['Lidar general inputs']['Yaw error'],                          
                              pitch_error = Qlunc_yaml_inputs['Components']['Lidar general inputs']['Pitch error'],                          
                              roll_error  = Qlunc_yaml_inputs['Components']['Lidar general inputs']['Roll error'],
                              dataframe   ={})                          


Created new lidar general inputs: Lidar_gral_inputs_Yaml


#### Atmospheric scenarios:
We can create different atmospheric scenarios under which the lidar uncertainties are assessed. Thus, single or multiple scenarios can be evaluated either including single or time-seried atmospheric variables values.

In [15]:
# Instantiating atmospheric scenario(s):
Atmospheric_TimeSeries = Qlunc_yaml_inputs['Atmospheric_inputs']['TimeSeries']  
                                                                           
if Atmospheric_TimeSeries:
    Atmos_TS_FILE           = '../metadata/AtmosphericData/'+Qlunc_yaml_inputs['Atmospheric_inputs']['Atmos_TS_FILE']
    AtmosphericScenarios_TS = pd.read_csv(Atmos_TS_FILE,delimiter=';',decimal=',')
    Atmospheric_inputs = {
                          'temperature' : list(AtmosphericScenarios_TS.loc[:,'T']),    
                          'humidity'    : list(AtmosphericScenarios_TS.loc[:,'H']),    
                          'rain'        : list(AtmosphericScenarios_TS.loc[:,'rain']),
                          'fog'         : list(AtmosphericScenarios_TS.loc[:,'fog']),
                          'time'        : list(AtmosphericScenarios_TS.loc[:,'t'])     
                          } 
    Atmospheric_Scenario = atmosphere(name        = 'Atmosphere1',
                                      temperature = Atmospheric_inputs['temperature'])
else:    

    Atmospheric_Scenario = atmosphere(name        = 'Atmosphere1',
                                      temperature = Qlunc_yaml_inputs['Atmospheric_inputs']['Temperature'])

Created new atmosphere: Atmosphere1


#### Lidar digital twin:
Puting all objects together like a puzzle we made up a lidar (digital twin). This enable assessing lidar uncertainties before a lidar is manufactured:

In [16]:
# Instantiating lidar device:
Lidar = lidar(name         = Qlunc_yaml_inputs['Lidar']['Name'],                       # Introduce the name of your lidar device.
              photonics    = eval(Qlunc_yaml_inputs['Lidar']['Photonics module']),     # Introduce the name of your photonics module.
              optics       = eval(Qlunc_yaml_inputs['Lidar']['Optics module']),        # Introduce the name of your optics module.
              power        = eval(Qlunc_yaml_inputs['Lidar']['Power module']),         # Introduce the name of your power module. NOT IMPLEMENTED YET!
              lidar_inputs = eval(Qlunc_yaml_inputs['Lidar']['Lidar inputs']),         # Introduce lidar general inputs
              unc_func     = eval(Qlunc_yaml_inputs['Lidar']['Uncertainty function'])) # Function estimating lidar global uncertainty


Created new lidar device: Caixa1


### **Calculating errors**:
Now we can ask for the error we are interested in. The modularity of the code allows us to ask separately either for a specific component or module uncertainty, or the global lidar uncertainty. 

As an example, we can ask for the optical amplifier uncertainty contribution. The uncertainty value is stored in *OpticalAmplifier_uncertainty*:

In [17]:
OpticalAmplifier_uncertainty,OpticalAmplifier_df = Lidar.photonics.optical_amplifier.Uncertainty(Lidar,Atmospheric_Scenario,cts,Qlunc_yaml_inputs)
# print(OpticalAmplifier_df)
print(OpticalAmplifier_uncertainty)

> c:\users\fcosta\swe_local\git_qlunc\uq_functions\uq_photonics_classes.py(154)UQ_Optical_amplifier()
    152         NF_w = 10**(FigureNoise/10) # Noise figure in watts
    153     pdb.set_trace()
--> 154     G_w  = 10**(Lidar.photonics.optical_amplifier.OA_Gain/10) # Gain in Watts
    155     # ASE noise:
    156     UQ_Optical_amplifier    = [np.array([10*np.log10((NF_w-(1/G_w))*cts.h*(cts.c/Lidar.lidar_inputs.Wavelength)*Lidar.photonics.optical_amplifier.OA_BW*G_w)]*len(Atmospheric_Scenario.temperature))]

ipdb> 
ipdb> 
ipdb> c


AttributeError: 'optical_amplifier' object has no attribute 'OA_BW'

In [ ]:
Photonics_Uncertainty,Photonics_df = Lidar.photonics.Uncertainty(Lidar,Atmospheric_Scenario,cts,Qlunc_yaml_inputs)
print(Photonics_df)

Or,  eventually we can ask for the lidar global uncertainty, where uncertainties coming from the different lidar modules are taken into account and combined following GUM suggestions:

In [ ]:
Lidar_Uncertainty = Lidar.Uncertainty(Lidar,Atmospheric_Scenario,cts,Qlunc_yaml_inputs)
print(Lidar_Uncertainty)

## Parameters access - dot notation

If want to access some parameter, follow dot notation. It can be done in two ways:

Either invoking the lidar object instance:
> Lidar.optics.scanner.focus_dist

or the component specific instance:

> Scanner.focus_distance

Let's see some examples:

### *Focus distance*:

In [ ]:
print(Scanner.focus_dist)

In [ ]:
print(Lidar.optics.scanner.focus_dist)

### Optical circulator *insertion loss*:

In [ ]:
print(Optical_circulator.insertion_loss)

### Optical circulator *ID*:

In [ ]:
print(Optical_circulator.Optical_CirculatorID)

### Optical amplifier gain

In [ ]:
Lidar.photonics.optical_amplifier.OA_Gain